Model:
 - [XGB](https://www.kaggle.com/kumareshd/talkingdata-mobile-user-demographics/chicken-momos-plate-1/run/299504/code) [another one](https://www.kaggle.com/tilii7/talkingdata-mobile-user-demographics/xgboost-simple-starter-more-features/code) another [one](https://www.kaggle.com/zfturbo/talkingdata-mobile-user-demographics/xgboost-simple-starter/code) [XGBoost 2.389](https://www.kaggle.com/sglink/talkingdata-mobile-user-demographics/xgboost-simple-starter/run/294382/code)
 - [optimizing parameters](https://www.kaggle.com/tilii7/talkingdata-mobile-user-demographics/bayesian-optimization-xgboost-parameters/comments)
 - [bag of apps](https://www.kaggle.com/hulkbulk/talkingdata-mobile-user-demographics/low-ram-bag-of-apps-python-2-7)
and [here](https://www.kaggle.com/xiaoml/talkingdata-mobile-user-demographics/low-ram-bag-of-apps-python) and [here](https://www.kaggle.com/xiaoml/talkingdata-mobile-user-demographics/bag-of-app-id-python-2-27392)
 - [apps labels linear](https://www.kaggle.com/dvasyukova/talkingdata-mobile-user-demographics/a-linear-model-on-apps-and-labels)
 - [brand and model](https://www.kaggle.com/dvasyukova/talkingdata-mobile-user-demographics/brand-and-model-based-benchmarks/comments) and [here](https://www.kaggle.com/autoptr/talkingdata-mobile-user-demographics/learning-pandas-follow-by-experts-i/run/310854/notebook)
 - [look at the script](https://www.kaggle.com/munitech4u/talkingdata-mobile-user-demographics/first-trial/run/302356)
 - https://www.kaggle.com/anokas/talkingdata-mobile-user-demographics/beat-the-benchmark-2-42107/discussion
    
https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22410/best-single-model
https://www.kaggle.com/chechir/talkingdata-mobile-user-demographics/keras-on-labels-and-brands
https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22269/python-code-for-evalulating-score
https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22269/python-code-for-evalulating-score
https://www.kaggle.com/zeroblue/talkingdata-mobile-user-demographics/better-log-loss-function/notebook

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn import linear_model, svm, neighbors, ensemble, naive_bayes

df_train = pd.DataFrame.from_csv("train.csv")
df_test  = pd.DataFrame.from_csv("test.csv")

In [2]:
df_train.drop(['device_id', 'phone_brand', 'device_model', 'gender', 'age'], axis=1, inplace=True)
df_test.drop(['device_id', 'phone_brand', 'device_model'], axis=1, inplace=True)

In [3]:
le = LabelEncoder() # e.classes_ to see all classes
le.fit(df_train['group'])
df_train['group'] = le.transform(df_train['group'])

In [4]:
df_train['lng'].fillna(0, inplace=True)
df_train['lat'].fillna(0, inplace=True)
df_test['lng'].fillna(0, inplace=True)
df_test['lat'].fillna(0, inplace=True)

### First attempt. ML without any prior thoughts
Try to do many algorithms on unmodified data

In [5]:
X = df_train.values[:,:-1]
Y = df_train.values[:,-1]

X_ = df_test.values

In [6]:
def score(clf):
    ## usage score((ensemble.GradientBoostingClassifier(n_estimators=300))
    pred = np.zeros((Y.shape[0], len(le.classes_)))
    for itrain, itest in StratifiedKFold(Y, n_folds=5, shuffle=True, random_state=0):
        Xtr, Xte = X[itrain, :], X[itest, :]
        ytr, yte = Y[itrain],    Y[itest]
        clf.fit(Xtr, ytr)
        pred[itest,:] = clf.predict_proba(Xte)

        print "\t{:.5f}".format(log_loss(yte, pred[itest,:]))

    return log_loss(Y, pred)

Few things tried without out of the box:
 - AdaBoostClassifier **2.47973**, 30 seconds
 - GradientBoostingClassifier **2.36487** 25 minutes
 - SVC rbf **2.42297** 7.5 hours
 - SVC sigmoid **2.42787** 7.5 hours
 - SVC nu **2.42668** 7.5 hours
 - everything else does not yield any reasonable results

When tried to do some tuning:
 - AdaBoostClassifier with higher number of estimators 300, 500, and smaller learning rate 0.5, 0.1 didn't help (~5 mins). 
 - GradientBoostingClassifier **2.3721** with n_estimators 300 took 70 mins and with 500 took **2.3827** took 2 hours. Max_depth 5, 7 has not improved anything **2.4016**, **2.4590** but runs 2-3 hours.
 - the best logistic regression (L1 1) achieves **2.3714**